# Rowe/Rho statistics

Any error in PSF modelling will propagate into galaxy shapes, and thereby the shear estimate. The impact of the PSF modelling errors on two-point shear correlation functions are quantified by the so-called Rowe ($\rho$)-statistics.

$\rho_1(\theta) \equiv \left\langle \; \delta e^*_{PSF}({\bf \phi}) \, \delta e_{PSF}({\bf \phi}+{\bf \theta}) \; \right\rangle $

$\rho_2(\theta) \equiv \left\langle \; e^*_{PSF}({\bf \phi}) \, \delta e_{PSF}({\bf \phi}+{\bf \theta}) \; \right\rangle $

$\rho_3(\theta) \equiv \left\langle \; \left( e^*_{PSF}\frac{\delta T_{PSF}}{T_{PSF}}\right)({\bf \phi}) \,  \left( e_{PSF} \frac{\delta T_{PSF}}{T_{PSF}}\right) ({\bf \phi}+{\bf \theta}) \; \right\rangle $

$\rho_4(\theta) \equiv \left\langle \; \delta e^*_{PSF}({\bf \phi}) \,  \left( e_{PSF} \frac{\delta T_{PSF}}{T_{PSF}}\right) ({\bf \phi}+{\bf \theta}) \; \right\rangle $

$\rho_5(\theta) \equiv \left\langle \; e^*_{PSF}({\bf \phi}) \,  \left( e_{PSF} \frac{\delta T_{PSF}}{T_{PSF}}\right) ({\bf \phi}+{\bf \theta}) \; \right\rangle $

The ticket DM-23539 incorporates a calculation of these statistics in `pipe_analysis` so that they are made as a part of regular QA. An example `CmdLineTask` to quickly generate one such plot is given below

` coaddAnalysis.py /datasets/hsc/repo/rerun/RC/w_2020_03/DM-23121 --calib /datasets/hsc/repo/CALIB  --output /scratch/kannawad/testCoaddAnalysis/  --id tract=9813 filter=HSC-I  -c doWriteParquetTables=False doPlotPsfFluxSnHists=False doPlotFootprintNpix=False doPlotOverlaps=False  > /scratch/kannawad/testCoaddAnalysis/coaddAnalysis.log & `

Because this is (being) integrated into `pipe_analysis`, I simply present the relevant code snippets here, instead of a stand-alone Jupyter notebook. A big pull request to ```lsst-dm/pipe_analysis``` is coming up.

In [12]:
class RhoStatistics(object):
    """Functor to calculate Rho statistics for a given star catalog and psf model"""
    def __init__(self, column, psfColumn, **kwargs):
        self.column = column
        self.psfColumn = psfColumn
        
        self.e1Func = E1(self.psfColumn)
        self.e2Func = E2(self.psfColumn)
        self.e1ResidsFunc = E1Resids(self.column, self.psfColumn)
        self.e2ResidsFunc = E2Resids(self.column, self.psfColumn)
        self.SizeResidsFunc = PsfTraceSizeDiff(self.column, self.psfColumn) # output will be in percentage

        self.treecorr_kwargs = {'nbins':kwargs.get('nbins',10), 
                        'min_sep':kwargs.get('min_sep',0.5), 
                        'max_sep':kwargs.get('max_sep',10), 
                        'sep_units':kwargs.get('sep_units',"arcmin"), 
                        'verbose':kwargs.get('verbose',0)}
    
    def __call__(self, catalog):
        e1 = self.e1Func(catalog)
        e2 = self.e2Func(catalog)
        e1Res = self.e1ResidsFunc(catalog)
        e2Res = self.e2ResidsFunc(catalog)
        SizeRes = self.SizeResidsFunc(catalog)/100. # since output is in percentage
 
        isFinite = np.isfinite(e1Res) & np.isfinite(e2Res) & np.isfinite(SizeRes)
        e1 = e1[isFinite]
        e2 = e2[isFinite]
        e1Res = e1Res[isFinite]
        e2Res = e2Res[isFinite]
        SizeRes = SizeRes[isFinite]
        
        # Scale the SizeRes by ellipticities
        e1SizeRes = e1*SizeRes
        e2SizeRes = e2*SizeRes

        # Package the arguments to capture auto-/cross-correlations for Rowe statistics
        g_args = {1: (e1Res, e2Res, None, None),
                  2: (e1, e2, e1Res, e2Res),
                  3: (e1SizeRes, e2SizeRes, None, None),
                  4: (e1Res, e2Res, e1SizeRes, e2SizeRes),
                  5: (e1, e2, e1SizeRes, e2SizeRes) }

        ra = np.rad2deg(catalog["coord_ra"][isFinite])*60. # arcmin
        dec = np.rad2deg(catalog["coord_dec"][isFinite])*60. # arcmin
 
        radii = dict.fromkeys(range(1,6))
        xip = dict.fromkeys(range(1,6))
        xipErr = dict.fromkeys(range(1,6))

        # Pass the appropriate arguments to the correlator
        for rhoIndex in range(1,6):
            # RFC: Should I just implement the corr2 functionality here inline?
            xy = corr2(ra, dec, *(g_args[rhoIndex]), ra_units="arcmin", dec_units="arcmin", **self.treecorr_kwargs)
            radii[rhoIndex] = np.exp(xy.meanlogr)
            xip[rhoIndex] = xy.xip
            xipErr[rhoIndex] = np.sqrt(xy.varxi)

        return radii, xip, xipErr
    
def corr2(ra, dec, g1a, g2a, g1b, g2b, ra_units="degrees", dec_units="degrees", **treecorr_kwargs):
    """ Function to compute auto-/cross-correlations between atmost two shear fields.

        This is used to compute Rowe statistics, given the appropriate 'shear' fields.
    """

    xy = treecorr.GGCorrelation(**treecorr_kwargs)
    catA = treecorr.Catalog(ra=ra, dec=dec, g1=g1a, g2=g2a, ra_units=ra_units,
                               dec_units=dec_units)
    if g1b is None or g2b is None:
        # Calculate the auto-correlation
        xy.process(catA)
    else:
        catB = treecorr.Catalog(ra=ra, dec=dec, g1=g1b, g2=g2b, ra_units=ra_units,
                               dec_units=dec_units)
        # Calculate the cross-correlation
        xy.process(catA, catB)

    return xy

class E1(object):
    """Function to calculate e1 ellipticity for a given object"""
    def __init__(self, column, unitScale=1.0):
        self.column = column
        self.unitScale = unitScale

    def __call__(self, catalog):
        E1 = ((catalog[self.column + "_xx"] - catalog[self.column + "_yy"])/
                 (catalog[self.column + "_xx"] + catalog[self.column + "_yy"]))
        return np.array(E1)*self.unitScale

class E2(object):
    """Function to calculate e2 ellipticity for a given object"""
    def __init__(self, column, unitScale=1.0):
        self.column = column
        self.unitScale = unitScale

    def __call__(self, catalog):
        E2 = (2.0*catalog[self.column + "_xy"]/
                 (catalog[self.column + "_xx"] + catalog[self.column + "_yy"]))
        return np.array(E2)*self.unitScale

class E1Resids(object):
    """Functor to calculate e1 ellipticity residuals for a given object and psf model"""
    def __init__(self, column, psfColumn, unitScale=1.0):
        self.column = column
        self.psfColumn = psfColumn
        self.unitScale = unitScale

    def __call__(self, catalog):
        srcE1func = E1(self.column, self.unitScale)
        psfE1func = E1(self.psfColumn, self.unitScale)

        srcE1 = srcE1func(catalog)
        psfE1 = psfE1func(catalog)

        e1Resids = srcE1 - psfE1
        return e1Resids


class E2Resids(object):
    """Functor to calculate e2 ellipticity residuals for a given object and psf model"""
    def __init__(self, column, psfColumn, unitScale=1.0):
        self.column = column
        self.psfColumn = psfColumn
        self.unitScale = unitScale

    def __call__(self, catalog):
        srcE2func = E2(self.column, self.unitScale)
        psfE2func = E2(self.psfColumn, self.unitScale)

        srcE2 = srcE2func(catalog)
        psfE2 = psfE2func(catalog)

        e2Resids = srcE2 - psfE2
        return e2Resids

In all plots that follow, only the absolute value of the Rowe statistics are plotted. The sign is denoted by the marker fillstyle: 

   * Closed points correspond to positive values
   * Open points correspond to negative values.

First, I will compute the Rowe statistics for PSF stars.

![RhoStatistics](plot-t9813-HSC-I-Rho_calib_psf_used-corr.png) 
![hsmRho](plot-t9813-HSC-I-hsmRho_calib_psf_used-corr.png)

Now for all stars selected as 

`base_ClassificationExtendedness_value < 0.5`

![](plot-t9813-HSC-I-Rho_all_stars-corr.png)
![](plot-t9813-HSC-I-hsmRho_all_stars-corr.png)

How do the Rowe statistics look in a given visit?

![](plot-v1248-Rho_calib_psf_used-corr.png)
![](plot-v1248-hsmRho_calib_psf_used-corr.png)

![](plot-v1248-Rho_all_stars-corr.png)
![](plot-v1248-hsmRho_all_stars-corr.png)

The smaller these values are, the better.

Before moving on, we can compare these plots with what we get from another dataset. For comparison, I plot the similar quantities from the Year 1 release of Dark Energy Survey (Zuntz et al., 2018)

![DES](DES_Rowe.png)

### Some insights:

- $\rho_3$, $\rho_4$ and $\rho_5$ involve PSF size residuals, with $\rho_5$ being a first-order quantity in the residuals.

- The scale-independence and high amplitude of $\rho_5$ is related to a known problem - The PSF model happens to be larger than the PSF stars. 
  It is interesting that the trend is other way around for all stars, with similar amplitude.
  
- $\rho_5$ is similar in amplitude for PSF stars at the visit level. But it is worse for non-PSF stars. This is puzzling!
  
- The amplitude and weak scale-dependence of $\rho_1$ and $\rho_2$ suggest other problems with PSF ellipticity modelling. It's likely it's related to the problem above.

Before signing off, we look at another tract

![RhoStatistics](plot-t9615-HSC-I-Rho_calib_psf_used-corr.png) 
![hsmRho](plot-t9615-HSC-I-hsmRho_calib_psf_used-corr.png)

![RhoStatistics](plot-t9615-HSC-I-Rho_all_stars-corr.png) 
![hsmRho](plot-t9615-HSC-I-hsmRho_all_stars-corr.png)

For yet another tract, this time with HSC-Y filter

![RhoStatistics](plot-t9697-HSC-Y-Rho_calib_psf_used-corr.png) 
![hsmRho](plot-t9697-HSC-Y-hsmRho_calib_psf_used-corr.png)

![](plot-t9697-HSC-Y-Rho_all_stars-corr.png)
![](plot-t9697-HSC-Y-hsmRho_all_stars-corr.png)